Cyclingrace results

Cyclingrace is an amateaur road race series.
Each race is a separate event which gives points towards season standing.

In [1]:
import requests
import pandas as pd
import yaml

from collections import defaultdict
from copy import deepcopy

In [2]:
from cr_racing_results import (
    RaceResults,
)

In [3]:
with open('races-config.yaml') as fp:
    race_config = yaml.safe_load(fp)

In [4]:
from pathlib import Path


official_clusters = defaultdict(tuple)

clusters_dir = Path(race_config['clusters'])
for file in list(clusters_dir.iterdir()):
    if file.name.startswith('.'):
        continue
    cluster = file.name.strip('+')
    with open(file) as fp:
        official_clusters[cluster] = official_clusters[cluster] + tuple(fp.read().split('\n'))

In [5]:
def parse_fixed_list(fixed_list: str) -> dict[str, int]:
    parsed = {}
    for entry in fixed_list.split('\n'):
        n1, n2, year = entry.split()
        parsed[f'{n1} {n2}'] = int(year)
    return parsed

A = """Тихонин Евгений 1998
Березняк Александр 2001
Миланин Константин 1986
Морской Сергей 1977
Петроченко Михаил 1988
Рыбачук Андрей 1993
Холин Олег 1985
Христенко Максим 1993"""

B = """Усольцев Алексей 1999
Шарифуллин Эдуард 1989
Земляной Николай 1986
Катаев Дмитрий 1979
Корнев Борис 1956
Мазилов Сергей 1990
Савкин Павел 1975"""

C = """Горячев Вячеслав 1959
Домбровский Роман 1986
Лимаренко Олег 1979"""

F = """Александрова Виктория 1991
Большакова Апполинария 1996
Грицкова Оя 1980
Гуренкова Екатерина 1985
Захаркина Валерия 2001
Ингелевич Екатерина 1986
Капущова Ольга 1971
Кишова Алина 1988
Королёва Анастасия 1993
Крючкова Полина 1992
Ланцова Мария 1962
Ломовцева Мария 2001
Мартынова Елена 1973
Муратова Ольга 1988
Ноготкова Екатерина 1980
Полевая Юлия 1985
Рыбина Светлана 2006
Серикова Наталья 1988
Стрельцова Галина 1991
Флоринская Яна 1984
Черных Светлана 1992
Чунакова Наталья 1974
Шараева Анна 1999
Юрченко Александра 2007"""

fixed_racers = {
    'A': parse_fixed_list(A),
    'B': parse_fixed_list(B),
    'C': parse_fixed_list(C),
    'F': parse_fixed_list(F),
}

In [41]:
import os
from tqdm import tqdm


races = []
cluster_distribution = None
for race in tqdm(race_config['races']):
    race = RaceResults.from_config(race, cluster_distribution, official_clusters)
    if race.name == 'Садовое кольцо':
        for cluster, fixes in fixed_racers.items():
            for racer, right_year in fixes.items():
                race.group[cluster].loc[race.group[cluster].name == racer, 'year_of_birth'] = right_year
                race.tt.loc[race.tt.name == racer, 'year_of_birth'] = right_year
        race.clusters = race.update_clusters(race.group, race.tt, cluster_distribution, official_clusters)
    elif race.name == 'Верея':
        race.group[cluster].loc[race.group[cluster].name == 'Нечаев Виктор', 'year_of_birth'] = 1994
        race.tt.loc[race.tt.name == 'Нечаев Виктор', 'year_of_birth'] = 1994
    race.save(os.path.join('data', race.name))
    cluster_distribution = race.clusters
    races.append(race)
races.sort(key=lambda race: race.order)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [04:06<00:00, 82.08s/it]


In [42]:
cluster_standing = {}

for race in races:
    for cluster, race_points in race.get_race_points().items():
        if cluster in cluster_standing:
            cluster_standing[cluster] = pd.merge(cluster_standing[cluster], race_points, how='outer', on=('name', 'year_of_birth'))
        else:
            cluster_standing[cluster] = race_points

In [43]:
renamed_columns = {
    "name": "Гонщик",
    "year_of_birth": "Год рождения",
    "total": "Очки"
}

for cluster in cluster_standing:
    all_results = cluster_standing[cluster].set_index(['name', 'year_of_birth'])
    all_results['total'] = all_results.sum(axis='columns')
    cluster_standing[cluster] = all_results.sort_values('total', ascending=False)


In [44]:
writer = pd.ExcelWriter('data/current_standing.xlsx', engine = 'xlsxwriter')
for cluster in cluster_standing:
    cluster_standing[cluster].reset_index().to_excel(writer, sheet_name=cluster)

    data = cluster_standing[cluster].reset_index().rename(renamed_columns, axis='columns')
    data.index += 1
    data.to_csv(f'data/cluster_{cluster}.csv')
writer.close()

In [45]:
all_clusters = races[0].clusters.rename({'cluster': races[0].name}, axis=1)
for race in races[1:]:
    all_clusters = all_clusters.merge(race.clusters.rename({'cluster': race.name}, axis=1), how='outer', on=['name', 'year_of_birth'])

In [46]:
all_clusters.loc[(all_clusters['Верея'] != all_clusters['Дубна']) & ~pd.isna(all_clusters['Дубна']) & ~pd.isna(all_clusters['Верея'])]

,name,year_of_birth,Садовое кольцо,Верея,Дубна
79,Карпов Антон,1979,A,A,B
143,Дмитриев Андрей,1981,C,C,B
277,Сальников Роман,1980,A,A,B
941,Худиев Омар,1994,NaN,B,A


# Менялись ли кластеры между гонок

In [16]:
all_clusters = races[0].clusters.rename({'cluster': races[0].name}, axis=1)
for race in races[1:]:
    all_clusters = all_clusters.merge(race.clusters.rename({'cluster': race.name}, axis=1), how='outer', on=['name', 'year_of_birth'])

In [47]:
pd.concat([
    all_clusters.loc[(all_clusters['Дубна'] != all_clusters['Верея']) & ~all_clusters['Дубна'].isna() & ~all_clusters['Верея'].isna()],
    all_clusters.loc[(all_clusters['Садовое кольцо'] != all_clusters['Верея']) & ~all_clusters['Садовое кольцо'].isna() & ~all_clusters['Верея'].isna()],
    all_clusters.loc[(all_clusters['Дубна'] != all_clusters['Верея']) & ~all_clusters['Дубна'].isna() & ~all_clusters['Верея'].isna()]
]).drop_duplicates()

,name,year_of_birth,Садовое кольцо,Верея,Дубна
79,Карпов Антон,1979,A,A,B
143,Дмитриев Андрей,1981,C,C,B
277,Сальников Роман,1980,A,A,B
941,Худиев Омар,1994,NaN,B,A
22,Васильев Дмитрий,1986,B,A,A
54,Черников Роман,1991,C,B,B
64,Жарков Олег,1980,C,B,B
440,Суслонов Александр,1991,C,B,B
475,Козлов Сергей,1985,C,B,B
515,Нечаев Виктор,1984,C,B,B
